![](https://velog.velcdn.com/images/hyejinbeck/post/e69505c7-b75a-410d-861a-73956babf1d1/image.png)

![](https://velog.velcdn.com/images/hyejinbeck/post/ed682230-205b-4cc7-a6c9-6f5e94010bc0/image.png)

![](https://velog.velcdn.com/images/hyejinbeck/post/da5ed408-2078-46d8-a093-97d00d6dd326/image.png)

In [93]:
# 초기설치 및 불러오기
# !pip install -r requirements.txt      하고나서 주석처리 
import pandas as pd 
data = pd.read_csv('./Dataset/Dataset_04.csv')
data.info()
data.head()

#################### 문제 1 

# 한국인 = 한국데이터만 추출 
data.LOCATION.unique()      # 한국은 'KOR'
data['LOCATION'].unique()   # 동일한 기능 

q1 = data[data.LOCATION == 'KOR']
q1.info()

# 년도별 육류 소비량 합계 구하기 (pivot이용하거나 crosstab이용하거나)
pd.pivot_table(q1, index = 'TIME', values = 'Value', aggfunc = 'sum')
pd.pivot_table(q1, index = 'TIME', values = 'Value', aggfunc = 'sum').reset_index()
q1_tab = pd.pivot_table(q1, index = 'TIME', values = 'Value', aggfunc = 'sum').reset_index()

# 그냥 참고 = crosstab 이용할경우 
pd.crosstab(index=q1['TIME'], columns='Value', values=q1['Value'], aggfunc='sum').reset_index()

# TIME과 Value간의 상관분석 (value가 한국인의 년도별 육류 소비량 합계) 
q1_tab.corr().loc['TIME','Value']

# 상관계수를 반올림하여, 소수점 둘째자리까지 구하기 
round(q1_tab.corr().loc['TIME','Value'], 3)

#################### 문제 2 

# 한국 KOR, 일본 JPN 데이터 추출 
q2 = data[data.LOCATION.isin(['KOR','JPN'])]
q2

# 육류별 = 육류 종류 확인 
q2.SUBJECT.unique()    # array(['BEEF', 'PIG', 'POULTRY', 'SHEEP'], dtype=object)

# 육류별 소비량을 두 국가별로 연도기준 
q2_tab = pd.pivot_table(q2, index=['TIME','SUBJECT'], columns = 'LOCATION')
q2_tab
q2_tab = pd.pivot_table(q2, index=['TIME','SUBJECT'], columns = 'LOCATION', values='Value')
q2_tab

# 대응표본 t 검정 준비 = 반드시 데이터의 갯수 동일 = 들어가는 순서 동일해야함 
q2_tab = q2_tab.dropna()    # 결측치 제거 = 그래야 쌍이 맞음 

# 대응표본 t 검정 함수
from scipy.stats import ttest_rel

# 대응표본 t 검정 진행 
ttest_rel(q2_tab['KOR'], q2_tab['JPN'])
q2_out = ttest_rel(q2_tab['KOR'], q2_tab['JPN'])
print(q2_out)
q2_out.statistic
print(f'{q2_out.statistic:.3f}')

#################### 문제 3 

# 한국만 포함한 데이터만 추출 
q3 = data[data.LOCATION == 'KOR']

# 육류 종류(그룹)별 회귀분석할 함수
from sklearn.linear_model import LinearRegression

# ######################################## sample 
# # TIME을 독립변수x로, Value를 종속변수y로하여 육류종류 SUBJECT 별로 회귀분석 설계 = 결정계수 추출 
# temp_s = q3[q3.SUBJECT == 'BEEF']  # 일단 육류종류 SUBJECT가 BEEF인것만 
# temp_s

# # model_error = LinearRegression().fit(temp['TIME'], temp['Value']) 
# # error 해석 : using array.reshape(-1, 1)

# # 회귀분석 모델 진행 
# # 차원을 바꿔야함 
# # temp[['TIME']]
# model_s = LinearRegression().fit(temp_s[['TIME']], temp_s['Value']) 

# # 회귀분석 결과값 = 결정계수값 확인 
# model_s.score(temp_s[['TIME']], temp_s['Value'])  # 0.835455805530312 

# # 육류 종류(그룹)별 회귀계수와 예측값 구하기 
# pred_s = model_s.predict(temp_s[['TIME']])

# # 다시 회귀분석 결과값 
# r2_s = model_s.score(temp_s[['TIME']], temp_s['Value']) 
# print(r2_s)

# # 가장 높은 결정계수를 가진 모델의 학습오차 중 MAPE 구하기 
# (temp_s['Value']-pred_s) / temp_s['Value']
# # temp['Value'] 는 양수 혹은 음수가 될 수 있어서 절대값을 취해야함. 
# ( abs(temp_s['Value'] - pred_s) / temp_s['Value'])
# ( abs(temp_s['Value'] - pred_s) / temp_s['Value']).sum()
# ( abs(temp_s['Value'] - pred_s) / temp_s['Value']).sum() * 100 
# ( abs(temp_s['Value'] - pred_s) / temp_s['Value']).sum() * 100  / len(temp_s)
# mape_s = ( abs(temp_s['Value'] - pred_s) / temp_s['Value']).sum() * 100  / len(temp_s)

######################################## 

# 근데 이제 육류 종류(그룹)별 구해야되서 다시 

sub_list = q3.SUBJECT.unique()

model = LinearRegression().fit(temp[['TIME']], temp['Value'])         


q3_out = []
for i in sub_list : 
    temp = q3[q3.SUBJECT == i ]       # i 는 육류 종류별 
    pred = model.predict(temp[['TIME']])
    r2 = model.score(temp[['TIME']], temp['Value']) 
    mape = ( abs(temp['Value'] - pred) / temp['Value']).sum() * 100  / len(temp)
    q3_out.append([i,r2,mape])
print(q3_out)
q3_out

# MAPE 학습 오차 (성능평가지표) 
q3_out = pd.DataFrame(q3_out, columns = ['sub','r2','mape']) # append 값 들어갔던 순서대로 columns 명 지정 
q3_out.sort_values
q3_out.sort_values('r2', ascending=False).head(1)  # r2기준으로 내림차순해서 가장 맨 위에 있는 값 = MAPE 가장 큰 값 

q3_answer = q3_out.sort_values('r2', ascending=False).head(1)
q3_answer

# 반올림하여 소수점 둘쨰자리까지 
round(q3_answer, 3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6718 entries, 0 to 6717
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   LOCATION  6718 non-null   object 
 1   SUBJECT   6718 non-null   object 
 2   TIME      6718 non-null   int64  
 3   Value     6718 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 210.1+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, 442 to 587
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   LOCATION  146 non-null    object 
 1   SUBJECT   146 non-null    object 
 2   TIME      146 non-null    int64  
 3   Value     146 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 5.7+ KB
TtestResult(statistic=5.662768873304346, pvalue=7.731889627475099e-08, df=145)
5.663
[['BEEF', -22.123008265285986, 97.93612798008533], ['PIG', -13.801405280165593, 99.19188261824794], ['POULTRY', -1

,sub,r2,mape
3,SHEEP,0.004,33.348


# 강의 정답과 다르다! 
마지막 정답 q3_out.sort_values('r2', ascending=False).head(1) 하게되면 <br>
강의에서는 POULTRY(sub), 0.9514(r2), 5.783(mape)가 나온다. <br>
내가 잘못 작성한 부분이 있는 것 같다. <br> 
다시 강의를 반복들으며, 빠지거나 오입력된 부분이 있는지 찾아보겠다. 

# 위의 코드에 틀린점 찾았다! 
3번문제 육류별 결정계수 구하는 반복문에서 
for문 안에 model= 모델링 파트를 넣지 않았다! 

# 다시 처음부터 시작해본다

In [ ]:
# 불러오기
# !pip install -r requirements.txt
import pandas as pd
data = pd.read_csv('./Dataset/Dataset_04.csv')
data.head()
data.info()

# 문제1

In [64]:
# TIME(x)과 Value(y) 상관분석을 위한, 한국 KOR 데이터만 추출
q1 = data[data.LOCATION == 'KOR']
q1 

,LOCATION,SUBJECT,TIME,Value
442,KOR,BEEF,1991,5.085481
443,KOR,BEEF,1992,5.685434
444,KOR,BEEF,1993,5.325765
445,KOR,BEEF,1994,5.698819
446,KOR,BEEF,1995,6.568429
...,...,...,...,...
583,KOR,SHEEP,2022,0.197951
584,KOR,SHEEP,2023,0.199098
585,KOR,SHEEP,2024,0.200266
586,KOR,SHEEP,2025,0.201451


In [66]:
# TIME(x)과 Value(y) 상관분석을 위한, 년도별 육류 소비량 합계 구하기 (pivot or crosstab) 
pd.pivot_table(q1, index='TIME', values='Value', aggfunc='sum')

,Value
TIME,
1990,5.774255
1991,21.911207
1992,26.901769
1993,26.938255
1994,27.877341
1995,30.042035
1996,31.928922
1997,32.002991
1998,30.009057


In [67]:
pd.pivot_table(q1, index='TIME', values='Value', aggfunc='sum').reset_index()

,TIME,Value
0,1990,5.774255
1,1991,21.911207
2,1992,26.901769
3,1993,26.938255
4,1994,27.877341
5,1995,30.042035
6,1996,31.928922
7,1997,32.002991
8,1998,30.009057
9,1999,35.383138


In [68]:
# TIME(x)과 Value(y) 상관분석을 위한, 상관분석 
q1_tab.corr()

,TIME,Value
TIME,1.000000,0.960124
Value,0.960124,1.000000


In [69]:
# TIME(x)과 Value(y) 상관분석을 위한, 상관계수구하기 
q1_tab.corr().loc['TIME','Value']

0.9601244896033333

In [72]:
# TIME(x)과 Value(y) 상관분석을 위한, 상관계수의 소수점 둘째자리까지 구하기 (셋째자리에서 반올림)
q1_tab.corr().loc['TIME','Value']

0.9601244896033333

In [73]:
q1_answer = q1_tab.corr().loc['TIME','Value']
round(q1_answer,3)

0.96

# 문제2

In [74]:
# 한국 KOR과 일본 JPN  데이터 추출 
q2 = data[data.LOCATION.isin(['KOR','JPN'])]
q2

,LOCATION,SUBJECT,TIME,Value
294,JPN,BEEF,1990,6.111035
295,JPN,BEEF,1991,6.517055
296,JPN,BEEF,1992,6.766244
297,JPN,BEEF,1993,7.406407
298,JPN,BEEF,1994,8.129090
...,...,...,...,...
583,KOR,SHEEP,2022,0.197951
584,KOR,SHEEP,2023,0.199098
585,KOR,SHEEP,2024,0.200266
586,KOR,SHEEP,2025,0.201451


In [75]:
# 두 국가의 각 연도기준으로 육류별 소비량 비교 
pd.pivot_table(q2, index = ['TIME','SUBJECT'], columns = 'LOCATION',values='Value')

LOCATION            JPN        KOR
TIME SUBJECT                      
1990 BEEF      6.111035        NaN
     PIG      13.219006        NaN
     POULTRY  12.844865   5.570102
     SHEEP     0.756696   0.204153
1991 BEEF      6.517055   5.085481
...                 ...        ...
2025 SHEEP     0.125060   0.201451
2026 BEEF      6.844875  10.737425
     PIG      15.544018  29.544063
     POULTRY  14.768340  17.907865
     SHEEP     0.122136   0.202656

[148 rows x 2 columns]

In [77]:
# NaN값 확인 = 두 국가 데이터를 비교하는 것이기 떄문에 하나라도 결측치가 있으면 비교가 불가능하여, 생략해야함 
q2_tab = q2_tab.dropna()
q2_tab

LOCATION            JPN        KOR
TIME SUBJECT                      
1990 POULTRY  12.844865   5.570102
     SHEEP     0.756696   0.204153
1991 BEEF      6.517055   5.085481
     PIG      13.239267   9.930182
     POULTRY  12.055422   6.738203
...                 ...        ...
2025 SHEEP     0.125060   0.201451
2026 BEEF      6.844875  10.737425
     PIG      15.544018  29.544063
     POULTRY  14.768340  17.907865
     SHEEP     0.122136   0.202656

[146 rows x 2 columns]

In [78]:
# 대응표본t검정 함수 
from scipy.stats import ttest_rel

In [79]:
# 대응표본t검정 진행 
ttest_rel(q2_tab['KOR'], q2_tab['JPN'])

TtestResult(statistic=5.662768873304346, pvalue=7.731889627475099e-08, df=145)

In [80]:
# 검정통계량 구하기
q2_out.statistic

5.662768873304346

In [81]:
# 검정통계량의 소숫점 셋째짜리 구하기 (넷째자리에서 반올림) 
q2_answer = q2_out.statistic
round(q2_answer,3)

5.663

# 문제3

In [82]:
# 한국데이터 추출 
q3 = data[data.LOCATION == 'KOR']

In [83]:
# 회귀분석 함수 
from sklearn.linear_model import LinearRegression

In [85]:
# 육류종류 확인 
sub_list = q3.SUBJECT.unique()
sub_list

array(['BEEF', 'PIG', 'POULTRY', 'SHEEP'], dtype=object)

In [86]:
# 한국의 TIME,Value의 육류종류(SUBJECT)별 회귀분석 및 결정계수의 MAPE를 위한, 육류종류별 회귀분석 (반복문) 
# 참고 
# temp['TIME'] 하게되면 차원이 맞지않아 error 
# temp[['TIME']] 하면 차원변경된다. 

temp['TIME']               # 데이터프레임 X 
temp['TIME'].shape         # (37,) 
temp['TIME'].ndim          # 1차원 

temp[['TIME']]             # 데이터프레임 O 
temp[['TIME']].shape       # (37, 1) 
temp[['TIME']].ndim        # 2차원 

temp['Value'].ndim         # 1차원 
temp[['Value']].ndim       # 2차원 

2

In [87]:
# 육류종류별 회귀분석 (반복문) 
q3_out = []
for i in sub_list : 
    temp=q3[q3.SUBJECT==i]
    model = LinearRegression().fit(temp[['TIME']], temp['Value'])   # 모델링 할땐 x=[[2치원으로]] ,y=1차원 
    pred = model.predict(temp[['TIME']])                            # 예측값 = 추정값 
    r2 = model.score(temp[['TIME']], temp['Value'])                 # 결정계수값 
    mape = (abs(temp['Value'] - pred) / temp['Value']).sum() * 100 / len(temp) # (절대값(y-예측값))/
#          절대값(---------실제값 ----예측값)                            # 양수,음수 될수 있어서 절대값 취해야함 
#                                       -----실제값      의 합계  * 100 / 지금사용한 데이터의 갯수(36개의 rows)
    q3_out.append([i, r2, mape])    
    
q3_out

[['BEEF', 0.835455805530312, 7.310893225858995],
 ['PIG', 0.9406814424683864, 5.634979881649262],
 ['POULTRY', 0.9514978845444608, 5.783357902874552],
 ['SHEEP', 0.004082992795654605, 33.3479658130492]]

In [88]:
# MAPE 학습 오차 (성능평가지표) 
q3_out = pd.DataFrame(q3_out, columns = ['sub','r2','mape'])   # 위에 append 들어간 순서대로 지정 
q3_out

,sub,r2,mape
0,BEEF,0.835456,7.310893
1,PIG,0.940681,5.634980
2,POULTRY,0.951498,5.783358
3,SHEEP,0.004083,33.347966


In [89]:
# MAPE 학습 오차 (성능평가지표) 데이터 정렬 
q3_out.sort_values('r2', ascending=False)     # 데이터 정렬 = r2결정계수값 기준으로 내림차순 = 가장 높은게 맨 위로 

,sub,r2,mape
2,POULTRY,0.951498,5.783358
1,PIG,0.940681,5.634980
0,BEEF,0.835456,7.310893
3,SHEEP,0.004083,33.347966


R2가 가장 큰 MAPE  = 5.783358 